Fuzzy String Matching

In [2]:
from __future__ import annotations
import itertools
from fuzzywuzzy import fuzz
import pandas as pd

Example pd.DataFrame with strings:

In [3]:
yh_typos_list = ['tellowhat',
'gellowhat',
'hellowhat',
'uellowhat',
'7ellowhat',
'6ellowhat',
'ywllowhat',
'ysllowhat',
'ydllowhat',
'yrllowhat',
'y4llowhat',
'y3llowhat',
'yeklowhat',
'yeplowhat',
'yeolowhat',
'yelkowhat',
'yelpowhat',
'yeloowhat',
'yelliwhat',
'yellkwhat',
'yelllwhat',
'yellpwhat',
'yell0what',
'yell9what',
'yelloqhat',
'yelloahat',
'yelloshat',
'yelloehat',
'yello3hat',
'yello2hat',
'yellowgat',
'yellowbat',
'yellownat',
'yellowjat',
'yellowuat',
'yellowyat',
'yellowhzt',
'yellowhst',
'yellowhwt',
'yellowhqt',
'yellowhar',
'yellowhaf',
'yellowhag',
'yellowhay',
'yellowha6',
'yellowha5',
'ellowhat',
'yllowhat',
'yelowhat',
'yelowhat',
'yellwhat',
'yellohat',
'yellowat',
'yellowht',
'yellowha',
'eyllowhat',
'ylelowhat',
'yellowhat',
'yelolwhat',
'yellwohat',
'yellohwat',
'yellowaht',
'yellowhta',
'yyellowhat',
'yeellowhat',
'yelllowhat',
'yelllowhat',
'yelloowhat',
'yellowwhat',
'yellowhhat',
'yellowhaat',
'yellowhatt']

yh_typos = (pd.DataFrame(yh_typos_list, columns = ['string'])
 .reset_index()
 .rename(columns = {"index": "id"})
)

In [4]:
def find_near_matches(df: pd.DataFrame,
                      similarity_pct_threshold: float = 90,
                      partial_similarity_pct_threshold: Optional[float] = None) -> pd.DataFrame:
    """Finds near/fuzzy matches among strings found by OCR in construction document. 

    To find exact matches only, set similarity_pct_threshold to 100 and
    partial_similarity_pct_threshold to None.

    Args:
        df: a pd.DataFrame containing an 'id' column and a 'string' column with
            strings found by OCR in construction document
        similarity_pct_threshold: The minimum Levenshtein distance similarity 
            ratio (scale is to 100) required for a pair of strings to be included
            in the output. This is based on the minimum number of single-character 
            edits (insertions, deletions or substitutions) required to change 
            one string into the other.
        partial_similarity_pct_threshold: Optionally, the minimum
            partial-string-adjusted Levenshtein distance similarity ratio required 
            for a pair of strings to be included in the output. Only one of the two
            thesholds needs to be met for a pair to be included. If supplied and 
            the two strings vary in length (and if the shorter string is length m, 
            and the longer string is length n), this will calculate the similarity 
            score of the best-matching length-m substring within the longer string. 

    Returns:
        A pd.DataFrame with columns for the IDs, contents, Levenshtein distance
        similarity ratios, and (if partial_similarity_pct_threshold is supplied)
        the partial-string-adjusted Levenshtein distance similarity ratios for
        each pair of strings that meets at least one threshold supplied.
    """

    return (pd.DataFrame(itertools.combinations((df["id"].values), 2))
    .join(df.set_index('id'), on = 0)
    .join(df.set_index('id'), on = 1, rsuffix = '2')
    .assign(ratio=lambda a: a.apply(lambda b: fuzz.ratio(b['string'], b['string2']), axis=1))
    .pipe(lambda df_: df_.assign(partial_ratio=lambda a: a.apply(lambda b: fuzz.partial_ratio(b['string'], b['string2']), axis=1)) if (partial_similarity_pct_threshold is not None) else df_)
    .loc[lambda r: (r["ratio"] >= similarity_pct_threshold) | ((partial_similarity_pct_threshold is not None) and (r['partial_ratio'] >= partial_similarity_pct_threshold))]
    .rename(columns = {0: 'id1', 1: 'id2', 'string': 'string1'})
    )

Examples:

In [5]:
display(find_near_matches(yh_typos, similarity_pct_threshold = 90, partial_similarity_pct_threshold = None))
display(find_near_matches(yh_typos, similarity_pct_threshold = 100, partial_similarity_pct_threshold = None)) #exact matches only
display(find_near_matches(yh_typos, similarity_pct_threshold = 90, partial_similarity_pct_threshold = 100))
display(find_near_matches(yh_typos, similarity_pct_threshold = 70.0, partial_similarity_pct_threshold = 100))

,id1,id2,string1,string2,ratio
45,0,46,tellowhat,ellowhat,94
115,1,46,gellowhat,ellowhat,94
184,2,46,hellowhat,ellowhat,94
252,3,46,uellowhat,ellowhat,94
319,4,46,7ellowhat,ellowhat,94
...,...,...,...,...,...
2551,68,70,yellowwhat,yellowhaat,90
2552,68,71,yellowwhat,yellowhatt,90
2553,69,70,yellowhhat,yellowhaat,90
2554,69,71,yellowhhat,yellowhatt,90


,id1,id2,string1,string2,ratio
2280,48,49,yelowhat,yelowhat,100
2535,65,66,yelllowhat,yelllowhat,100


,id1,id2,string1,string2,ratio,partial_ratio
45,0,46,tellowhat,ellowhat,94,100
115,1,46,gellowhat,ellowhat,94,100
184,2,46,hellowhat,ellowhat,94,100
252,3,46,uellowhat,ellowhat,94,100
319,4,46,7ellowhat,ellowhat,94,100
...,...,...,...,...,...,...
2551,68,70,yellowwhat,yellowhaat,90,90
2552,68,71,yellowwhat,yellowhatt,90,90
2553,69,70,yellowhhat,yellowhaat,90,90
2554,69,71,yellowhhat,yellowhatt,90,90


,id1,id2,string1,string2,ratio,partial_ratio
0,0,1,tellowhat,gellowhat,89,89
1,0,2,tellowhat,hellowhat,89,89
2,0,3,tellowhat,uellowhat,89,89
3,0,4,tellowhat,7ellowhat,89,89
4,0,5,tellowhat,6ellowhat,89,89
...,...,...,...,...,...,...
2551,68,70,yellowwhat,yellowhaat,90,90
2552,68,71,yellowwhat,yellowhatt,90,90
2553,69,70,yellowhhat,yellowhaat,90,90
2554,69,71,yellowhhat,yellowhatt,90,90
